In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import csv

In [2]:
def get_metrics(exp_dict, mic_idx, train, var_list, var2, metric, dataset_condition, comparison_domain, scenarios, model_type):
    #Getting list of each case
    mix_metrics = []
    enh_metrics = []
    mvng_enh_metrics = []
    mvng_mix_metrics = []


    for var in var_list:
        for test in scenarios:

            if dataset_condition =="reverb":
                exp_app_str = f't60_{var}'
            elif dataset_condition =="noisy":
                exp_app_str = f'snr_{var}'
            elif dataset_condition =="noisy_reverb":
                exp_app_str = f't60_{var}_snr_{var2}'
            else:
                exp_app_str = ''

            if model_type == "miso":
                key_str = f'train_{train}_mic_idx_{mic_idx}_test_{test}_{exp_app_str}' if comparison_domain!="doa" else f'train_{train}_test_{test}_{exp_app_str}'
            elif model_type =="mimo":
                key_str = f'train_{train}_test_{test}_{exp_app_str}'
            
            metrics = exp_dict[key_str]._asdict()
            start_phrase = "est" if comparison_domain =="doa" else "enh"
            
            if test=="stationary":
                mix_metrics.append(metrics[f'mix_{metric}'])
                enh_metrics.append(metrics[f'{start_phrase}_{metric}'])
            elif test=="moving":
                mvng_mix_metrics.append(metrics[f'mix_{metric}'])
                mvng_enh_metrics.append(metrics[f'{start_phrase}_{metric}'])
            else:
                break

            #print(f'{t60}, {test}, {exp_dict[key_str]}')

    if metric =="stoi" or metric == "frm_Acc" or metric == "blk_Acc":
        stoi_format= lambda lst: [ round(float(x)*100, 2) for x in lst]

        mix_metrics, enh_metrics, mvng_metrics, mvng_mix_metrics = stoi_format(mix_metrics), stoi_format(enh_metrics), stoi_format(mvng_enh_metrics), stoi_format(mvng_mix_metrics)

    else:
        pesq_format= lambda lst: [ round(float(x), 2) for x in lst]

        mix_metrics, enh_metrics, mvng_metrics, mvng_mix_metrics = pesq_format(mix_metrics), pesq_format(enh_metrics), pesq_format(mvng_enh_metrics), pesq_format(mvng_mix_metrics)


    return mix_metrics, enh_metrics, mvng_metrics, mvng_mix_metrics

In [3]:
def get_var_list(dataset_condition):
    if dataset_condition=="noisy_reverb":
        snr_list = [-5.0, 0.0, 5.0, 10.0, 20.0] # col
        t60_list = [0.2, 0.4, 0.6, 0.8, 1.0] # rows
        return snr_list, t60_list
    elif dataset_condition=="noisy":
        snr_list = [-5.0, -2.5, 0.0, 2.5, 5.0, 10.0, 15.0, 20.0] # col
        t60 = 0
        return t60, snr_list
    elif dataset_condition=="reverb":
        snr = None
        t60_list = [0.0, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
        return snr, t60_list

In [4]:
scenarios = ["stationary", "moving"]
#comparison_domain=""
se_metrics_lst = ["stoi", 'pesq_nb']
doa_metrics_lst = ["frm_Acc", "blk_Acc"]
from collections import namedtuple

metrics = namedtuple('metrics', ["mix_stoi", "enh_stoi", "mix_pesq_nb", "enh_pesq_nb", "mix_frm_Acc", "est_frm_Acc", "mix_blk_Acc", "est_blk_Acc"])

In [14]:
dataset_condition = "reverb"
train = 'stationary'
mic_idx=-1
model_type = "mimo"
file_name = "mimo_ph_diff_loss_unweighted_results"
exp_dict_mimo = torch.load(f'../Logs/{dataset_condition}/{dataset_condition}_mimo_ph_diff_loss_v2_results.pt')
if dataset_condition !="noisy_reverb":
    var2, var_list = get_var_list(dataset_condition)

header = ['Train', 'Metric', 0.0, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
f = open(f"{file_name}.csv", 'w') 
write = csv.writer(f)
write.writerow(header)
for train in ["stationary", "moving"]:
    for comparison_domain in ["se", "doa",]:
        _metrics = se_metrics_lst if comparison_domain=="se" else doa_metrics_lst
        for metric in _metrics:
            mix_mimo_metric, enh_mimo_metric, mvng_mimo_metric, mvng_mix_mimo_metric = get_metrics(exp_dict_mimo, mic_idx=-1, train=train, var_list=var_list, var2=var2, metric=metric, dataset_condition=dataset_condition, comparison_domain=comparison_domain, scenarios=scenarios,model_type="mimo")
            print(f'{metric}, mix_mimo_metric: {mix_mimo_metric.insert(0, f"mix_{metric}")}, enh_mimo_metric: {enh_mimo_metric.insert(0, f"enh_{metric}")}, mvng_mimo_metric: {mvng_mimo_metric.insert(0, f"mvng_enh_{metric}")}, mvng_mix_mimo_metric: {mvng_mix_mimo_metric.insert(0, f"mvng_mix_{metric}")}\n')
            print(f'{metric}, mix_mimo_metric: {mix_mimo_metric.insert(0, f"{train}")}, enh_mimo_metric: {enh_mimo_metric.insert(0, f"{train}")}, mvng_mimo_metric: {mvng_mimo_metric.insert(0, f"{train}")}, mvng_mix_mimo_metric: {mvng_mix_mimo_metric.insert(0, f"{train}")}\n')
            rows = [ mix_mimo_metric, enh_mimo_metric, mvng_mimo_metric, mvng_mix_mimo_metric]
            #print(f"rows: {rows}" )
            #writing to csv file
            write.writerows(rows)


f.close()

stoi, mix_mimo_metric: None, enh_mimo_metric: None, mvng_mimo_metric: None, mvng_mix_mimo_metric: None

stoi, mix_mimo_metric: None, enh_mimo_metric: None, mvng_mimo_metric: None, mvng_mix_mimo_metric: None

pesq_nb, mix_mimo_metric: None, enh_mimo_metric: None, mvng_mimo_metric: None, mvng_mix_mimo_metric: None

pesq_nb, mix_mimo_metric: None, enh_mimo_metric: None, mvng_mimo_metric: None, mvng_mix_mimo_metric: None

frm_Acc, mix_mimo_metric: None, enh_mimo_metric: None, mvng_mimo_metric: None, mvng_mix_mimo_metric: None

frm_Acc, mix_mimo_metric: None, enh_mimo_metric: None, mvng_mimo_metric: None, mvng_mix_mimo_metric: None

blk_Acc, mix_mimo_metric: None, enh_mimo_metric: None, mvng_mimo_metric: None, mvng_mix_mimo_metric: None

blk_Acc, mix_mimo_metric: None, enh_mimo_metric: None, mvng_mimo_metric: None, mvng_mix_mimo_metric: None

stoi, mix_mimo_metric: None, enh_mimo_metric: None, mvng_mimo_metric: None, mvng_mix_mimo_metric: None

stoi, mix_mimo_metric: None, enh_mimo_metric: 